<h3>This notebook contains training code of a neural network performing the "Region of Interest" (ROI) detection work. Its output will be used at later stages to preprocess images being fed to the actual classifier. ROI NN will detect bounding boxes surrounding lesions and another algorithm will crop the images so that the aspect ratio required by InceptionResNetV2 NN is used.</h3>

In [1]:
import os
import absl.logging
import PIL.Image

import numpy as np
import pandas as pd
import tensorflow as tf

from typing import Callable
from tensorflow import keras

absl.logging.set_verbosity(absl.logging.ERROR)

In [2]:
get_names = lambda root_path: [
    file_name.split('.')[0]
    for dir_path, _, file_names in os.walk(root_path)
    for file_name in file_names
]
get_paths = lambda path: [f'{os.path.join(root, file)}' for root, dirs, files in os.walk(path) for file in files]
base_dir = os.path.join('..', 'data', 'images_original_inception_resnet_v2_200x150_splitted')
train_dir = os.path.join(base_dir, 'training')
valid_dir = os.path.join(base_dir, 'validation')

In [3]:
def get_images_array(paths: list[str]) -> np.ndarray:
    rows = []
    rescale = keras.layers.Rescaling(1./255)

    for path in paths:
        with PIL.Image.open(path) as image:
            image_array = np.asarray(image)
            rescaled_image = rescale(image_array)
            rows.append(rescaled_image)

    return np.array(rows)


train_paths = get_paths(train_dir)
valid_paths = get_paths(valid_dir)
X_train = get_images_array(train_paths)
X_valid = get_images_array(valid_paths)

In [4]:
SMALLER_WIDTH = 600 // 3
SMALLER_HEIGHT = 450 // 3

In [5]:
train_names = set(get_names(train_dir))
valid_names = set(get_names(valid_dir))
metadata_path = os.path.join('..', 'data', 'HAM10000_metadata_ext.csv')
data = pd.read_csv(metadata_path).sort_values(by='image_id')
relevant_cols = ['left', 'top', 'right', 'bottom']
train_df = data[data['image_id'].isin(train_names)].sort_values(by='image_id')[relevant_cols]
valid_df = data[data['image_id'].isin(valid_names)].sort_values(by='image_id')[relevant_cols]
ys_train = train_df.to_numpy().astype(float)
ys_train[:, [0, 2]] /= SMALLER_HEIGHT
ys_train[:, [1, 3]] /= SMALLER_WIDTH
ys_valid = valid_df.to_numpy().astype(float)
ys_valid[:, [0, 2]] /= SMALLER_HEIGHT
ys_valid[:, [1, 3]] /= SMALLER_WIDTH

Using simple IoU loss made the model get stuck - it wasn't getting any better or worse. Using CIoU makes the model start from a worse starting point but it's learning. Let's see how it does after 1000 epochs.

Implementation idea: [Medium article](https://medium.com/analytics-vidhya/different-iou-losses-for-faster-and-accurate-object-detection-3345781e0bf)
*IoU formulas and more: [Researchgate article](https://www.researchgate.net/figure/CIoU-calculation-conceptualized-The-classification-function-L-cls-only-penalizes-if_fig5_355427005)

In [6]:
def ciou_loss(y_true: tf.Tensor, y_pred: tf.Tensor) -> tf.Tensor:
    """
    Parameters:
    y_true -- ground truth bounding box, tensor of shape (?, 4), [xmin, ymin, xmax, ymax]
    y_pred -- predicted bounding box, tensor of shape (?, 4)

    Returns:
    ciou_loss -- scalar loss, tensor of shape ()
    """
    true_x1, true_y1, true_x2, true_y2 = tf.split(y_true, 4, axis = -1)
    pred_x1, pred_y1, pred_x2, pred_y2 = tf.split(y_pred, 4, axis = -1)
    # intersection calculation:
    # take leftmost x coord and rightmost x coord, subtract to get the width and constraint with
    # 0 to avoid negative values, do the same with ys
    intersect_w = tf.maximum(0.0, tf.minimum(true_x2, pred_x2) - tf.maximum(true_x1, pred_x1))
    intersect_h = tf.maximum(0.0, tf.minimum(true_y2, pred_y2) - tf.maximum(true_y1, pred_y1))
    # calculate area
    intersection = intersect_w * intersect_h
    # calculate areas of the predicted and actual bounding box
    # then calculate the union
    true_area = (true_x2 - true_x1) * (true_y2 - true_y1)
    pred_area = (pred_x2 - pred_x1) * (pred_y2 - pred_y1)
    union = true_area + pred_area - intersection
    # IoU calculation
    iou = intersection / (union + 1e-9)  # Adding epsilon to avoid division by zero
    # distance between the box centers
    true_center_x = (true_x1 + true_x2) / 2
    true_center_y = (true_y1 + true_y2) / 2
    pred_center_x = (pred_x1 + pred_x2) / 2
    pred_center_y = (pred_y1 + pred_y2) / 2
    # from pythagorean theorem - calculate euclidean distance a^2 + b^2 = c^2
    # here the distance is still in the square units, because it doesn't matter in this
    # context since if z1 < z2 then sqrt(z1) < sqrt(z2)
    # we can save some (very little) computation time by omitting the root calculation
    center_distance = tf.square(true_center_x - pred_center_x) + tf.square(true_center_y - pred_center_y)
    # enclosing box
    enclose_x1 = tf.minimum(true_x1, pred_x1)
    enclose_y1 = tf.minimum(true_y1, pred_y1)
    enclose_x2 = tf.maximum(true_x2, pred_x2)
    enclose_y2 = tf.maximum(true_y2, pred_y2)
    enclose_w = enclose_x2 - enclose_x1
    enclose_h = enclose_y2 - enclose_y1
    # CIoU term
    ciou_term = (1 - iou) + center_distance / (tf.square(enclose_w) + tf.square(enclose_h) + 1e-9)

    return tf.reduce_mean(ciou_term)


def ciou_metric(y_true: tf.Tensor, y_pred: tf.Tensor) -> tf.Tensor:
    ciou_term = ciou_loss(y_true, y_pred)

    return 1.0 - tf.reduce_mean(ciou_term) # higher value is better

In [7]:
def get_model(loss: Callable, metric: Callable) -> keras.Model:
    def get_conv_module(prev: keras.layers.Layer, filters: int, kernel_size: int) -> keras.layers.Layer:
        x = keras.layers.Conv2D(filters, kernel_size, padding='same', activation='relu')(prev)
        x = keras.layers.Conv2D(filters * 2, kernel_size, padding='same', activation='relu')(x)
        x = keras.layers.MaxPooling2D()(x)

        return x


    _input = keras.layers.Input(shape=(SMALLER_HEIGHT, SMALLER_WIDTH, 3))
    conv_module = get_conv_module(_input, 16, 7)
    conv_module = get_conv_module(conv_module, 32, 5)
    conv_module = get_conv_module(conv_module, 64, 3)
    conv_module = keras.layers.Flatten()(conv_module)
    locator_module = keras.layers.Dense(512, activation='relu')(conv_module)
    locator_module = keras.layers.Dense(256, activation='relu')(locator_module)
    locator_module = keras.layers.Dense(128, activation='relu')(locator_module)
    locator_module = keras.layers.Dense(4, activation='sigmoid')(locator_module)

    model = keras.Model(_input, locator_module)

    model.compile(optimizer='adam', loss=loss, metrics=[metric])

    print(model.summary())

    return model

In [8]:
def run_model(model_factory, model_name: str, loss: Callable, metric: Callable):
    early_stopping = keras.callbacks.EarlyStopping(monitor='val_iou_metric', patience=10,
                                                   min_delta=1e-6)
    reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                                                  patience=5, min_lr=0.001)
    model_checkpoint = keras.callbacks.ModelCheckpoint(
        filepath='models/' + model_name + '_{epoch}',
        save_best_only=True)
    tensor_board = keras.callbacks.TensorBoard(log_dir=f'tensor_logs/{model_name}')
    model = model_factory(loss, metric)

    return model.fit(
        X_train,
        ys_train,
        validation_data=(X_valid, ys_valid),
        epochs=500,
        batch_size=64,
        callbacks=[reduce_lr, model_checkpoint, tensor_board])

In [ ]:
history = run_model(get_model, f'roi_detection_iou_2', ciou_loss, ciou_metric)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 200, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 150, 200, 16)      2368      
                                                                 
 conv2d_1 (Conv2D)           (None, 150, 200, 32)      25120     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 75, 100, 32)      0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 75, 100, 32)       25632     
                                                                 
 conv2d_3 (Conv2D)           (None, 75, 100, 64)       51264     
                                                             

INFO:tensorflow:Assets written to: models\roi_detection_iou_2_1\assets


126/126 [==============================] - 38s 234ms/step - loss: 0.5283 - ciou_metric: 0.4718 - val_loss: 0.4519 - val_ciou_metric: 0.5483 - lr: 0.0010
Epoch 2/500
 32/126 [======>.......................] - ETA: 18s - loss: 0.4696 - ciou_metric: 0.5304